In [ ]:
#We are using the datasets library from HUgging face. Feel free to skip this block if you already have it installed
!pip install datasets

In [ ]:
#In order to download and merge the two corpora that is Stanford natural language inference and multi-genre natural language inference corpora

import datasets

s_nli = datasets.load_dataset('snli', split = 'train')

s_nli


In [ ]:
print(s_nli[0])

{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


In [ ]:
#loading the dataset in this format

m_nli = datasets. load_dataset('glue', 'mnli', split = 'train')

m_nli
print(m_nli[0])

In [ ]:
#the idx column is not useful for us as thhe stanford dataset do not have that column. and in order for us to merge the dataset, they mush have the same name and number of columns

m_nli = m_nli.remove_columns(['idx'])
s_nli = s_nli.cast(m_nli.features)
dataset = datasets.concatenate_datasets([s_nli, m_nli])

In [ ]:
print(len(dataset))
# there are -1 values in the label feature, these are where no class could be decided so we remove
dataset = dataset.filter(
    lambda x: 0 if x['label'] == -1 else 1
)
print(len(dataset))

In [ ]:
!pip install sentence-transformers # uncomment this if you need to install sentence transformers

from sentence_transformers import InputExample 
from tqdm.auto import tqdm

training_examples = []
for row in tqdm(dataset): 
  training_examples.append(InputExample(
      texts=[row['premise'], row['hypothesis']],
      label = row['label']
  ))

In [ ]:
from torch.utils.data import DataLoader as dl

batch_size = 16
loader = dl(training_examples, batch_size= batch_size, shuffle = True)

In [ ]:
from sentence_transformers import models, SentenceTransformer

bert_model = models.Transformer('bert-base-uncased')

pooler = models.Pooling(
    bert_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)
model = SentenceTransformer(modules= [bert_model,pooler])
model

In [ ]:
from sentence_transformers import losses

loss = losses.SoftmaxLoss(
    model = model,
    sentence_embedding_dimension= model.get_sentence_embedding_dimension(),
     num_labels=3)

In [ ]:
epochs = 1
warm_steps = int(len(loader) * epochs *0.1)

model.fit(
    train_objectives=[(loader,loss)], 
    epochs = epochs,
    warmup_steps=warm_steps,
    output_path='./sbert_test_b',
    show_progress_bar=False
)

In [ ]:
 sentences = [
    "the fifty mannequin heads floating in the pool kind of freaked them out",
    "she swore she just saw her sushi move",
    "he embraced his new life as an eggplant",
    "my dentist tells me that chewing bricks is very bad for your teeth",
    "the dental specialist recommended an immediate stop to flossing with construction materials",
    "i used to practice weaving with spaghetti three hours a day",
    "the white water rafting trip was suddenly halted by the unexpected brick wall",
    "the person would knit using noodles for a few hours daily",
    "it was always dangerous to drive with him since he insisted the safety cones were a slalom course",
    "the woman thinks she saw her raw fish and rice change position"
]

In [ ]:
embeds = model.encode(sentences)

In [ ]:
import numpy as np
from sentece_transformers import cos_sim as cs

def simX(model):
  sim = n.zeros((len(sentences), len(sentences)))
  for i in range(len(sentences)):
    sim[i:,i] = cs(embeds[i], embeds[i:])

  return sim

In [ ]:
#OR

#sim = np.zeros((len(sentences), len(sentences)))

#for i in range(len(sentences)):
 #   sim[i:,i] = cs(embeddings[i], embeddings[i:])

#sim

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sim = simX(model)

sns.heatmap(sim, annot=True)